In [1]:
!pip install -q "transformers>=4.39.3" datasets accelerate bitsandbytes>=0.41.2 peft sentence-transformers faiss-cpu


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [2]:
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"  # same model family the paper used :contentReference[oaicite:0]{index=0}&#8203;:contentReference[oaicite:1]{index=1}


In [ ]:
import json, random, re, string

def letter(n):          # 0 -> A, 1 -> B …
    return string.ascii_uppercase[n]

def make_line(q, choices, ans_idx):
    # Build a letter-labeled choice list, keep ground-truth answer explicit
    joined = "; ".join(f"{letter(i)}) {c}" for i, c in enumerate(choices))
    answer = choices[ans_idx]
    return f"<BOS> Q: {q} Choices: {joined}. Correct: {answer} <EOS>"

with open("training_data.json") as fp:
    data = json.load(fp)

with open("train.txt", "w", encoding="utf-8") as f:
    for row in data:
        f.write(make_line(row["question"], row["choices"], row["answer"]) + "\n")

print("Lines:", sum(1 for _ in open("train.txt")))


Lines: 3456


In [3]:
!pip install -q huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `colab-mistral` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab-mistr

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tok.pad_token = tok.eos_token   # needed for batch collation

ds = load_dataset("text", data_files={"train":"train.txt"})["train"]

def tok_fn(batch):
    return tok(batch["text"], truncation=True, max_length=512)
ds = ds.map(tok_fn, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

In [ ]:
from transformers import BitsAndBytesConfig    # <- correct import
import torch, bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)


lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [ ]:
import bitsandbytes, transformers
print(bitsandbytes.__version__)   # should show 0.42.x or 0.43.x
print(transformers.__version__)   # 4.39+ is fine


0.45.5
4.51.3


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tok,
        mlm=False            # causal LM, not masked LM
)

bsz = 4                      # per-device
g_accum = 8                  # 4×8 = 32 “virtual” batch
lr  = 2e-5                   # mid-range LR from the paper
EPOCHS = 3

args = TrainingArguments(
    output_dir="ft-mistral",
    per_device_train_batch_size=bsz,
    gradient_accumulation_steps=g_accum,
    num_train_epochs=EPOCHS,
    learning_rate=lr,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    fp16=True,
    logging_strategy="steps",    # <-- tell HF *when* to log
    logging_steps=25,            # already set
    logging_first_step=True,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(model=model, args=args, train_dataset=ds, data_collator=data_collator)
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,3.008800
25,2.410800
50,1.381900
75,1.079000
100,1.075000
125,1.046900
150,1.034600
175,1.011500
200,1.025300
225,1.013900


TrainOutput(global_step=324, training_loss=1.160960974516692, metrics={'train_runtime': 4039.2152, 'train_samples_per_second': 2.567, 'train_steps_per_second': 0.08, 'total_flos': 4.353401250545664e+16, 'train_loss': 1.160960974516692, 'epoch': 3.0})

In [ ]:
trainer.save_model("ft-mistral")
tok.save_pretrained("ft-mistral")

('ft-mistral/tokenizer_config.json',
 'ft-mistral/special_tokens_map.json',
 'ft-mistral/tokenizer.model',
 'ft-mistral/added_tokens.json',
 'ft-mistral/tokenizer.json')

In [ ]:
!zip -r ft-mistral.zip ft-mistral

updating: ft-mistral/ (stored 0%)
updating: ft-mistral/checkpoint-216/ (stored 0%)
updating: ft-mistral/checkpoint-216/special_tokens_map.json (deflated 73%)
updating: ft-mistral/checkpoint-216/trainer_state.json (deflated 68%)
updating: ft-mistral/checkpoint-216/tokenizer_config.json (deflated 68%)
updating: ft-mistral/checkpoint-216/README.md (deflated 66%)
updating: ft-mistral/checkpoint-216/tokenizer.model (deflated 55%)
updating: ft-mistral/checkpoint-216/training_args.bin (deflated 52%)
updating: ft-mistral/checkpoint-216/scheduler.pt (deflated 56%)
updating: ft-mistral/checkpoint-216/tokenizer.json (deflated 85%)
updating: ft-mistral/checkpoint-216/scaler.pt (deflated 60%)
updating: ft-mistral/checkpoint-216/adapter_model.safetensors (deflated 7%)
updating: ft-mistral/checkpoint-216/adapter_config.json (deflated 54%)
updating: ft-mistral/checkpoint-216/optimizer.pt (deflated 8%)
updating: ft-mistral/checkpoint-216/rng_state.pth (deflated 25%)
updating: ft-mistral/special_tokens_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
q = "When was the first asteroid Ceres discovered?"
prompt = f"Q: {q}\nA:"
out = model.generate(**tok(prompt, return_tensors="pt").to(model.device),
                     max_new_tokens=40)
print(tok.decode(out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q: When was the first asteroid Ceres discovered?
A: January 1, 1801


In [ ]:
import shutil, pathlib

src  = pathlib.Path("ft-mistral.zip")
dest = pathlib.Path("/content/drive/MyDrive/models/mistral/ft-mistral.zip")
dest.parent.mkdir(parents=True, exist_ok=True)

shutil.copy2(src, dest)    # copy with metadata
print("Copied to", dest)

Copied to /content/drive/MyDrive/models/mistral/ft-mistral.zip


In [4]:
# (If you see more than one GPU and want to stay on a single card)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [5]:
from google.colab import files, drive
# files.upload()                              # interactive picker
drive.mount('/content/drive')                 # if the zip lives in Drive
zip_path = "/content/drive/MyDrive/models/mistral/ft-mistral.zip"   # adjust if needed

!unzip -q "$zip_path" -d /content/ft-mistral
ckpt_dir = "/content/ft-mistral"              # we'll reuse this variable
!ls -1 $ckpt_dir                              # quick look; should list adapter_model.safetensors


Mounted at /content/drive
ft-mistral


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "mistralai/Mistral-7B-Instruct-v0.2"        # original weights

bnb_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,            # change to float16 if bf16 unsupported
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2",
        quantization_config=bnb_conf,
        torch_dtype=torch.bfloat16,
        device_map={"": 0},
        trust_remote_code=True,
)


# ❷  attach your LoRA deltas
model = PeftModel.from_pretrained(base, "/content/ft-mistral/ft-mistral")
tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

model.eval()
print("Model & LoRA loaded on", next(model.parameters()).device)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Model & LoRA loaded on cuda:0


In [9]:
import json, pathlib

val_path = "/content/mmlu_astronomy_validation.json"   # adjust if needed
test_path = "/content/mmlu_astronomy_test.json"

with open(val_path)  as fp: val_rows  = json.load(fp)
with open(test_path) as fp: test_rows = json.load(fp)

print(f"validation  rows: {len(val_rows)}")
print(f"test rows:        {len(test_rows)}")


validation  rows: 16
test rows:        152


In [10]:
import torch, tqdm
from math import log

def logp_answer(prompt, answer_text):
    """Token-level log-probability of a *full* answer string."""
    with torch.inference_mode():
        inp   = tokenizer(prompt, return_tensors="pt").to("cuda")
        label = tokenizer(" "+answer_text, return_tensors="pt").input_ids.to("cuda")
        joined = torch.cat([inp.input_ids, label[:, :-1]], dim=-1)   # predict each token
        logits = model(joined).logits[:, -label.size(1):, :]
        logp   = torch.nn.functional.log_softmax(logits, dim=-1)
        return logp.gather(2, label.unsqueeze(-1)).sum().item()

def mc_accuracy(rows):
    correct = 0
    for row in tqdm.tqdm(rows):
        prompt  = f"Q: {row['question']}\nA:"
        scores  = [logp_answer(prompt, c) for c in row['choices']]
        pred_ix = max(range(len(scores)), key=lambda i: scores[i])
        correct += (pred_ix == row['answer'])
    return correct / len(rows)

val_acc  = mc_accuracy(val_rows)
test_acc = mc_accuracy(test_rows)

print(f"Validation accuracy: {val_acc:.2%}")
print(f"Test accuracy:       {test_acc:.2%}")


100%|██████████| 152/152 [05:58<00:00,  2.36s/it]

Validation accuracy: 43.75%
Test accuracy:       42.76%


In [10]:
q = "When was the first asteroid Ceres discovered?"
prompt = f"Q: {q}\nA:"
out = model.generate(**tok(prompt, return_tensors="pt").to(model.device),
                     max_new_tokens=40)
print(tok.decode(out[0], skip_special_tokens=True))